In [2]:
import fitz  # PyMuPDF
import logging
from pypdf import PdfReader, PdfWriter
logger = logging.getLogger(__name__)


In [5]:
image_file_path = r"C:\Users\burag\VercelDaisler\daisler\api\files\drops_image.jpg"
pdf_file_path = r"C:\Users\burag\VercelDaisler\daisler\api\files\drops_image.pdf"
not_centered_pdf_file_path = r"C:\Users\burag\VercelDaisler\daisler\api\files\not_centered_drops_image.pdf"
cutline_pdf = r"C:\Users\burag\VercelDaisler\daisler\api\cut line.pdf"
desert_pdf = r"C:\Users\burag\VercelDaisler\daisler\api\files\desert.pdf"

In [4]:
def read_pdf_with_fitz(file_path):
    try:
        pdf = fitz.open(file_path)
        return pdf
    except Exception as exc:
        logger.exception("read_pdf_with_fitz failed: %s", exc)
        raise exc


In [5]:
def read_pdf_with_pypdf(file_path):
    try:
        reader = PdfReader(file_path)
        return reader
    except Exception as exc:
        logger.exception("read_pdf_with_pypdf failed: %s", exc)
        raise exc


In [ ]:
pypdf_pdf = read_pdf_with_pypdf(pdf_file_path)
fitz_pdf = read_pdf_with_fitz(pdf_file_path)

In [12]:
from PIL import Image
import io

desert_pdf = read_pdf_with_fitz(desert_pdf)
desert_pdf_page = desert_pdf.load_page(0)  # first page
#let's show only the images from the first page in this notebook
images = desert_pdf_page.get_images(full=True)
# Extract all images from the first page and composite them as they appear on the page
canvas = Image.new("RGB", (desert_pdf_page.rect.width, desert_pdf_page.rect.height), "white")

for img in images:
    xref = img[0]
    pix = fitz.Pixmap(desert_pdf, xref)
    bbox = desert_pdf_page.get_image_bbox(img)
    pil_img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    # Resize if needed to fit the bbox
    pil_img = pil_img.resize((int(bbox.width), int(bbox.height)))
    canvas.paste(pil_img, (int(bbox.x0), int(bbox.y0)))
    pix = None

canvas.show()




TypeError: 'float' object cannot be interpreted as an integer

In [7]:
import pymupdf

doc = pymupdf.open(cutline_pdf)


In [10]:
# --- config ---
SPOT_NAME = "CutContour"          # name of the spot color
ALT_CMYK = [0.3, 0.5, 1, 0]       # alternate CMYK (magenta-like on screen)
HAIRLINE = True                   # True = hairline stroke (device pixel), False = fixed width
STROKE_WIDTH_PT = 0.5             # used if HAIRLINE is False

infile = pdf_file_path
outfile = "sssssssssss.pdf"

# Define your rectangle manually (x0, y0, x1, y1) in PDF points
RECT = (100, 100, 400, 400)

# --- open PDF ---
doc = fitz.open(infile)

# --- create Separation object for spot color ---
sep_obj = (
    f"[ /Separation /{SPOT_NAME} /DeviceCMYK "
    f"<< /FunctionType 2 /Domain [0 1] /C0 [0 0 0 0] "
    f"/C1 [{' '.join(map(str, ALT_CMYK))}] /N 1 >> ]"
)
sep_xref = doc.get_new_xref()
doc.update_object(sep_xref, sep_obj)

# --- first page only ---
page = doc[0]
page_xref = page.xref

# --- ensure /Resources exists ---
res_key = doc.xref_get_key(page_xref, "Resources")[1]
if not res_key:
    res_xref = doc.get_new_xref()
    doc.update_object(res_xref, "<<>>")
    doc.xref_set_key(page_xref, "Resources", f"{res_xref} 0 R")
else:
    if res_key.endswith(" R"):
        res_xref = int(res_key.split()[0])
    else:
        res_xref = doc.get_new_xref()
        doc.update_object(res_xref, res_key)
        doc.xref_set_key(page_xref, "Resources", f"{res_xref} 0 R")

# --- ensure /ColorSpace includes /CS1 ---
cs_key = doc.xref_get_key(res_xref, "ColorSpace")[1]
if not cs_key:
    cs_xref = doc.get_new_xref()
    doc.update_object(cs_xref, f"<< /CS1 {sep_xref} 0 R >>")
    doc.xref_set_key(res_xref, "ColorSpace", f"{cs_xref} 0 R")
else:
    if cs_key.endswith(" R"):
        cs_xref = int(cs_key.split()[0])
        current = doc.xref_object(cs_xref)
        if "obj" in current:
            current = current[current.find("<<"):current.rfind(">>")+2]
        if "/CS1" not in current:
            updated = current[:-2] + f" /CS1 {sep_xref} 0 R >>"
            doc.update_object(cs_xref, updated)
    else:
        cs_xref = doc.get_new_xref()
        base = cs_key.strip()
        if base.endswith(">>"):
            updated = base[:-2] + f" /CS1 {sep_xref} 0 R >>"
        else:
            updated = f"<< /CS1 {sep_xref} 0 R >>"
        doc.update_object(cs_xref, updated)
        doc.xref_set_key(res_xref, "ColorSpace", f"{cs_xref} 0 R")

# --- build rectangle content ---
x0, y0, x1, y1 = RECT
w = 0 if HAIRLINE else STROKE_WIDTH_PT
content = (
    "q\n"
    f"{w} w\n"
    "/CS1 CS\n"
    "1 SCN\n"
    f"{x0} {y0} {x1 - x0} {y1 - y0} re\n"
    "S\n"
    "Q\n"
).encode("ascii")

# --- add stream ---
stream_xref = doc.get_new_xref()
doc.update_object(stream_xref, "<<>>")
doc.update_stream(stream_xref, content)

cont_key = doc.xref_get_key(page_xref, "Contents")[1]
if not cont_key:
    doc.xref_set_key(page_xref, "Contents", f"{stream_xref} 0 R")
else:
    if cont_key.endswith(" R"):
        old_xref = int(cont_key.split()[0])
        arr_xref = doc.get_new_xref()
        doc.update_object(arr_xref, f"[ {old_xref} 0 R {stream_xref} 0 R ]")
        doc.xref_set_key(page_xref, "Contents", f"{arr_xref} 0 R")
    elif cont_key.strip().startswith("["):
        arr = cont_key.strip()
        updated = arr[:-1] + f" {stream_xref} 0 R ]"
        arr_xref = doc.get_new_xref()
        doc.update_object(arr_xref, updated)
        doc.xref_set_key(page_xref, "Contents", f"{arr_xref} 0 R")
    else:
        old_xref = doc.get_new_xref()
        doc.update_object(old_xref, "<<>>")
        doc.update_stream(old_xref, cont_key.encode("latin1"))
        arr_xref = doc.get_new_xref()
        doc.update_object(arr_xref, f"[ {old_xref} 0 R {stream_xref} 0 R ]")
        doc.xref_set_key(page_xref, "Contents", f"{arr_xref} 0 R")

# --- save ---
doc.save(outfile)
doc.close()
print(f"✅ Saved {outfile} with rectangle at {RECT}")





✅ Saved sssssssssss.pdf with rectangle at (100, 100, 400, 400)


'/Layer /MC0 BDC \nq\n0 500 500 -500 re\nW n\n1 1 1 rg\n/GS0 gs\nq 1 0 0 1 438.3727 236.7391 cm\n0 0 m\n0 -91.247 -73.97 -165.217 -165.217 -165.217 c\n-256.464 -165.217 -330.435 -91.247 -330.435 0 c\n-330.435 91.247 -256.464 165.217 -165.217 165.217 c\n-73.97 165.217 0 91.247 0 0 c\nf\nQ\n/CS0 CS 1  SCN\nq 1 0 0 1 438.3727 236.7391 cm\n0 0 m\n0 -91.247 -73.97 -165.217 -165.217 -165.217 c\n-256.464 -165.217 -330.435 -91.247 -330.435 0 c\n-330.435 91.247 -256.464 165.217 -165.217 165.217 c\n-73.97 165.217 0 91.247 0 0 c\nh\nS\nQ\nEMC \nQ\n

In [ ]:


# --- config ---
PAGE_W, PAGE_H = 500, 500
RECT = (60, 60, 440, 440)  # x0, y0, x1, y1
SPOT_NAME = "CutContour"   # or "CUT"
ALT_CMYK = [0.3, 0.5, 1, 0]    # alternate (magenta) so it's obvious on-screen
HAIRLINE = True            # True -> hairline (device pixel width), False -> set an explicit width (pt)
STROKE_WIDTH_PT = 0.5      # used only if HAIRLINE is False
OUTFILE = "cut_spot_rect_3.pdf"

doc = fitz.open()
page = doc.new_page(width=PAGE_W, height=PAGE_H)

# --- 1) Create a Separation colorspace object for the spot (/CutContour or /CUT) ---
# We'll define:
#   [/Separation /CutContour /DeviceCMYK << /FunctionType 2 /Domain [0 1] /C0 [0 0 0 0] /C1 [0 1 0 0] /N 1 >>]
# The tint 1.0 = 100% of the spot; alternate is CMYK ALT_CMYK for on-screen / non-spot viewers.

sep_obj = (
    f"[ /Separation /{SPOT_NAME} /DeviceCMYK "
    f"<< /FunctionType 2 /Domain [0 1] /C0 [0 0 0 0] /C1 [{' '.join(map(str, ALT_CMYK))}] /N 1 >> ]"
)

# Allocate a new xref for the Separation object and write it
sep_xref = doc.get_new_xref()           # low-level: create a new object number
doc.update_object(sep_xref, sep_obj)        # store its content

# --- 2) Ensure the page has a /Resources dict with a /ColorSpace entry that maps /CS1 -> sep_xref ---
page_xref = page.xref

# Get /Resources; if missing, create an empty dict
res_key = doc.xref_get_key(page_xref, "Resources")[1]
if not res_key:
    res_xref = doc.get_new_xref()
    doc.update_object(res_xref, "<<>>")
    doc.xref_set_key(page_xref, "Resources", f"{res_xref} 0 R")
else:
    # It’s a reference like "123 0 R" or an inline dict. Normalize to an xref.
    if res_key.endswith(" R"):
        res_xref = int(res_key.split()[0])
    else:
        # Inline dict: move it to its own object so we can edit reliably
        res_xref = doc.get_new_xref()
        doc.update_object(res_xref, res_key)
        doc.xref_set_key(page_xref, "Resources", f"{res_xref} 0 R")

# Fetch existing ColorSpace dict (if any)
cs_key = doc.xref_get_key(res_xref, "ColorSpace")[1]
if not cs_key:
    # no /ColorSpace: create one with /CS1 mapped to our Separation object
    cs_xref = doc.get_new_xref()
    doc.update_object(cs_xref, f"<< /CS1 {sep_xref} 0 R >>")
    doc.xref_set_key(res_xref, "ColorSpace", f"{cs_xref} 0 R")
else:
    # there is a /ColorSpace entry; make sure it is an indirect object we can edit
    if cs_key.endswith(" R"):
        cs_xref = int(cs_key.split()[0])
        # Read current dict content and append /CS1 if missing
        current = doc.xref_object(cs_xref)
        # Strip "obj ... endobj" wrapper if present
        if "obj" in current and "endobj" in current:
            current = current[current.find("<<"):current.rfind(">>")+2]
        # If /CS1 not present, add it
        if "/CS1" not in current:
            updated = current[:-2] + f" /CS1 {sep_xref} 0 R >>"
            doc.update_object(cs_xref, updated)
    else:
        # Inline dictionary — convert to indirect and add /CS1
        cs_xref = doc.get_new_xref()
        base = cs_key.strip()
        if base.endswith(">>"):
            updated = base[:-2] + f" /CS1 {sep_xref} 0 R >>"
        else:
            updated = f"<< /CS1 {sep_xref} 0 R >>"
        doc.update_object(cs_xref, updated)
        doc.xref_set_key(res_xref, "ColorSpace", f"{cs_xref} 0 R")

# --- 3) Append a small content stream that uses that spot color and draws the rectangle (stroke only) ---
x0, y0, x1, y1 = RECT
w = 0 if HAIRLINE else STROKE_WIDTH_PT

# PDF drawing:
# q               save graphics state
# {w} w          line width (0 = hairline)
# /CS1 CS        set stroking color space to our Separation
# 1 SCN          set stroking color (tint = 1.0 => 100% spot)
# x y width height re   rectangle path
# S               stroke
# Q               restore
# EMC             end marked content
content = (
    "/Layer /MC0 BDC\n"  # Begin marked content (optional, for print workflows)
    "q\n"
    f"{w} w\n"
    "/CS1 CS\n"
    "1 SCN\n"
    f"{x0} {y0} {x1 - x0} {y1 - y0} re\n"
    "S\n"
    "Q\n"
    "EMC\n"  # End marked content
).encode("ascii")

# Add this as an additional page content stream
stream_xref = doc.get_new_xref()
doc.update_object(stream_xref, "<<>>")
doc.update_stream(stream_xref, content)

# Attach the stream to the page's /Contents (preserving any existing content)
cont_key = doc.xref_get_key(page_xref, "Contents")[1]
if not cont_key:
    # No content yet — set directly
    doc.xref_set_key(page_xref, "Contents", f"{stream_xref} 0 R")
else:
    # If multiple or single, normalize to an array and append our stream
    if cont_key.endswith(" R"):
        # single stream -> make an array [old new]
        old_xref = int(cont_key.split()[0])
        arr_xref = doc.get_new_xref()
        doc.update_object(arr_xref, f"[ {old_xref} 0 R {stream_xref} 0 R ]")
        doc.xref_set_key(page_xref, "Contents", f"{arr_xref} 0 R")
    elif cont_key.strip().startswith("["):
        # inline array -> append
        arr = cont_key.strip()
        updated = arr[:-1] + f" {stream_xref} 0 R ]"
        arr_xref = doc.get_new_xref()
        doc.update_object(arr_xref, updated)
        doc.xref_set_key(page_xref, "Contents", f"{arr_xref} 0 R")
    else:
        # inline stream — rare; wrap it into an array with our new stream
        old_xref = doc.get_new_xref()
        doc.update_object(old_xref, "<<>>")  # <-- Add this line
        doc.update_stream(old_xref, cont_key.encode("latin1"))
        arr_xref = doc.get_new_xref()
        doc.update_object(arr_xref, f"[ {old_xref} 0 R {stream_xref} 0 R ]")
        doc.xref_set_key(page_xref, "Contents", f"{arr_xref} 0 R")

# Add a basic ExtGState if you want to use /GS0 gs (optional)
gs_dict = "<< /GS0 << /CA 1 /ca 1 >> >>"
gs_xref = doc.get_new_xref()
doc.update_object(gs_xref, gs_dict)
doc.xref_set_key(res_xref, "ExtGState", f"{gs_xref} 0 R")

# Save the PDF
doc.save(OUTFILE)
doc.close()

print(f"Done. Wrote {OUTFILE}")


Done. Wrote cut_spot_rect_3.pdf


In [ ]:
import fitz  # PyMuPDF library

def image_to_pdf(image_path, output_pdf_path):
    """
    Reads an image and creates a one-page PDF with the same dimensions as the image.

    Args:
        image_path (str): The path to the input image file.
        output_pdf_path (str): The path to save the output PDF file.
    """
    try:
        # Open the image file to get its dimensions
        image_pixmap = fitz.Pixmap(image_path)
        image_width = image_pixmap.width
        image_height = image_pixmap.height

        # Create a new PDF document with one page
        # The page dimensions are set to the image's dimensions
        doc = fitz.open()
        page = doc.new_page(width=image_width, height=image_height)

        # Insert the image onto the new page
        # rect defines the rectangle where the image will be placed
        # Here, it covers the entire page from (0,0) to (width, height)
        page.insert_image(rect=fitz.Rect(0, 0, image_width, image_height), filename=image_path)

        # Save the document to the specified output path
        doc.save(output_pdf_path)
        doc.close()

        print(f"Successfully created PDF at: {output_pdf_path}")

    except Exception as e:
        print(f"An error occurred: {e}")

# --- Example Usage ---
# Replace 'your_image.jpg' with the path to your actual image file
# and 'output_document.pdf' with your desired output PDF file name.
image_file = r"C:\Users\burag\Downloads\drops_image.jpg"
pdf_file = r"C:\Users\burag\Downloads\drops_image.pdf"
image_to_pdf(image_file, pdf_file)

Successfully created PDF at: C:\Users\burag\Downloads\drops_image.pdf


In [11]:
def add_cutline(doc, rect, spot_name="CutContour", alt_cmyk=(0.3, 0.5, 1, 0),
                page_num=0, hairline=True, stroke_width=0.5):
    """
    Add a rectangle cutline (spot color stroke) to a PDF page.

    Args:
        doc (fitz.Document): an open PDF document
        rect (tuple): (x0, y0, x1, y1) coordinates of the rectangle
        spot_name (str): name of the spot color (default: "CutContour")
        alt_cmyk (tuple): alternate CMYK values for screen display
        page_num (int): zero-based page index (default: 0 = first page)
        hairline (bool): True = hairline stroke, False = use stroke_width
        stroke_width (float): width in pt, used if hairline=False

    Returns:
        fitz.Document: the modified document
    """
    page = doc[page_num]
    page_xref = page.xref

    # --- create Separation object for spot color ---
    sep_obj = (
        f"[ /Separation /{spot_name} /DeviceCMYK "
        f"<< /FunctionType 2 /Domain [0 1] /C0 [0 0 0 0] "
        f"/C1 [{' '.join(map(str, alt_cmyk))}] /N 1 >> ]"
    )
    sep_xref = doc.get_new_xref()
    doc.update_object(sep_xref, sep_obj)

    # --- ensure /Resources exists ---
    res_key = doc.xref_get_key(page_xref, "Resources")[1]
    if not res_key:
        res_xref = doc.get_new_xref()
        doc.update_object(res_xref, "<<>>")
        doc.xref_set_key(page_xref, "Resources", f"{res_xref} 0 R")
    else:
        if res_key.endswith(" R"):
            res_xref = int(res_key.split()[0])
        else:
            res_xref = doc.get_new_xref()
            doc.update_object(res_xref, res_key)
            doc.xref_set_key(page_xref, "Resources", f"{res_xref} 0 R")

    # --- ensure /ColorSpace includes /CS1 ---
    cs_key = doc.xref_get_key(res_xref, "ColorSpace")[1]
    if not cs_key:
        cs_xref = doc.get_new_xref()
        doc.update_object(cs_xref, f"<< /CS1 {sep_xref} 0 R >>")
        doc.xref_set_key(res_xref, "ColorSpace", f"{cs_xref} 0 R")
    else:
        if cs_key.endswith(" R"):
            cs_xref = int(cs_key.split()[0])
            current = doc.xref_object(cs_xref)
            if "obj" in current:
                current = current[current.find("<<"):current.rfind(">>")+2]
            if "/CS1" not in current:
                updated = current[:-2] + f" /CS1 {sep_xref} 0 R >>"
                doc.update_object(cs_xref, updated)
        else:
            cs_xref = doc.get_new_xref()
            base = cs_key.strip()
            if base.endswith(">>"):
                updated = base[:-2] + f" /CS1 {sep_xref} 0 R >>"
            else:
                updated = f"<< /CS1 {sep_xref} 0 R >>"
            doc.update_object(cs_xref, updated)
            doc.xref_set_key(res_xref, "ColorSpace", f"{cs_xref} 0 R")

    # --- build rectangle content ---
    x0, y0, x1, y1 = rect
    w = 0 if hairline else stroke_width
    content = (
        "q\n"
        f"{w} w\n"
        "/CS1 CS\n"
        "1 SCN\n"
        f"{x0} {y0} {x1 - x0} {y1 - y0} re\n"
        "S\n"
        "Q\n"
    ).encode("ascii")

    # --- add stream ---
    stream_xref = doc.get_new_xref()
    doc.update_object(stream_xref, "<<>>")
    doc.update_stream(stream_xref, content)

    cont_key = doc.xref_get_key(page_xref, "Contents")[1]
    if not cont_key:
        doc.xref_set_key(page_xref, "Contents", f"{stream_xref} 0 R")
    else:
        if cont_key.endswith(" R"):
            old_xref = int(cont_key.split()[0])
            arr_xref = doc.get_new_xref()
            doc.update_object(arr_xref, f"[ {old_xref} 0 R {stream_xref} 0 R ]")
            doc.xref_set_key(page_xref, "Contents", f"{arr_xref} 0 R")
        elif cont_key.strip().startswith("["):
            arr = cont_key.strip()
            updated = arr[:-1] + f" {stream_xref} 0 R ]"
            arr_xref = doc.get_new_xref()
            doc.update_object(arr_xref, updated)
            doc.xref_set_key(page_xref, "Contents", f"{arr_xref} 0 R")
        else:
            old_xref = doc.get_new_xref()
            doc.update_object(old_xref, "<<>>")
            doc.update_stream(old_xref, cont_key.encode("latin1"))
            arr_xref = doc.get_new_xref()
            doc.update_object(arr_xref, f"[ {old_xref} 0 R {stream_xref} 0 R ]")
            doc.xref_set_key(page_xref, "Contents", f"{arr_xref} 0 R")

    return doc


In [13]:

doc = fitz.open(pdf_file_path)

# add rectangle on first page from (100,100) to (400,400)
page_dimensions = doc[0].rect
print("Page dimensions:", page_dimensions)
# add_cutline(doc, (100, 100, 400, 400))
add_cutline(doc, (30, 30, 866, 1570))

doc.save("final.pdf")
doc.close()


Page dimensions: Rect(0.0, 0.0, 896.0, 1600.0)
